# Using LSTMs to Forecast Monthly Milk Production

Time series datasets are defined as databases that contain a sequence of datapoints over time. This includes stock prices (e.g., price per day), weather (e.g., degrees Celsius per day), and sales figures (net profit by quarter), among others.

In this example, we'll use monthly milk production, which is stored as units of milk produced per month.

<div style="text-align: center;"> <img src = "res/model_building/lstms_milk_icon.jpg" width="25%"/> </div>

Unfortunately, neural networks have a difficult time with memory and variable input features. This was fixed by recurrent neural networks and later by long short-term memory networks (LSTMs). This makes them ideal for forecasting with time series.

<div style="text-align: center;"> <img src = "res/model_building/lstms_lstm_cell.jpg" width="50%"/> </div>

In this practicum, we'll be using LSTMs to predict monthly milk production. <strong> We'll be using a very high level implementation of LSTMs, thus, you won't need to know the details of how an LSTM cell works.</storng>

# 0 | Google Colab Setup

In [ ]:
import os
import shutil
import stat

In [ ]:
def copy_safe(src, dst, max_len=200):
    """Copy files, skip long paths"""
    skipped = 0
    for root, dirs, files in os.walk(src):
        rel_path = os.path.relpath(root, src)
        dst_root = os.path.join(dst, rel_path) if rel_path != '.' else dst
        if len(dst_root) < max_len:
            os.makedirs(dst_root, exist_ok=True)
            for file in files:
                dst_file = os.path.join(dst_root, file)
                if len(dst_file) < max_len:
                    try: shutil.copy2(os.path.join(root, file), dst_file)
                    except: skipped += 1
                else: skipped += 1
        else: skipped += len(files)
    return skipped

In [ ]:
# Setup resources if needed
setup_ran = False
if not os.path.exists('res'):
    print("Setting up resources...")
    setup_ran = True
    
    # Cleanup, clone, copy
    repo = 'deep_learning_resources'
    if os.path.exists(repo):
        shutil.rmtree(repo, onerror=lambda f,p,e: os.chmod(p, stat.S_IWRITE) or f(p))
    
    !git clone --depth=1 https://github.com/jjv31/deep_learning_resources
    
    if os.path.exists(f'{repo}/res'):
        skipped = copy_safe(f'{repo}/res', 'res')
        print(f"Setup complete! {'(' + str(skipped) + ' long filenames skipped)' if skipped else ''}")
    
    shutil.rmtree(repo, onerror=lambda f,p,e: os.chmod(p, stat.S_IWRITE) or f(p))

In [ ]:
# Only refresh if we just downloaded resources
if setup_ran:
    from IPython.display import Javascript, display
    import time
    
    print("Refreshing images...")
    
    # Try browser refresh + aggressive image reload
    display(Javascript(f'''
    try {{ setTimeout(() => window.location.reload(true), 2000); }} catch(e) {{}}
    
    const t = {int(time.time())};
    document.querySelectorAll('img').forEach((img, i) => {{
        if (img.src.includes('res/')) {{
            const src = img.src.split('?')[0];
            setTimeout(() => img.src = src + '?v=' + t + '_' + i, i * 50);
        }}
    }});
    '''))
    
    print("If images don't appear, press Ctrl+Shift+R to hard refresh!")
else:
    print("Resources already exist, skipping setup.")

# 1 | Loads & Inspects Dataset

### 1.0 | Install missing packages

In [ ]:
%pip install statsmodels pygame

### 1.1 | Imports

In [ ]:
import pandas as pd
import numpy as np

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Other
from statsmodels.tsa.seasonal import seasonal_decompose

# Preprocessing
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Neural Nets
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input, Flatten
from keras.optimizers import Adam
from keras import metrics

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### 1.2 | Aux functions. Just run

In [ ]:
#Function to facilitate evaluating our models
def print_score(clf, X, y_true):

    # Gets predicted labels
    if isinstance(clf, keras.models.Sequential): # If the model is a Keras neural network
        y_pred = (clf.predict(X) >= 0.5).astype(int) 
    else: # Normal scikit-learn model
        y_pred = clf.predict(X)

    # Gets key performance indicators
    accuracy = round(accuracy_score(y_true, y_pred), 4)
    recall = round(recall_score(y_true, y_pred), 4)
    precision = round(precision_score(y_true, y_pred), 4)
    f1 = round(f1_score(y_true, y_pred), 4)

    # Displays them
    print(f"F1 = {f1:.4f} | Recall = {recall* 100:.2f}% | Precision = {precision*100:.2f}%")

In [ ]:
# Plots the performance of the neural network
def plot_performance(training_values, validation_values, metric_name = "Recall"):

    epochs = range(1, len(training_values) + 1)
    
    sns.set() 
    plt.plot(epochs, training_values, '-', label=f'Training {metric_name}')
    plt.plot(epochs, validation_values, ':', label=f'Validation {metric_name}')

    plt.title(f'Training and Validation {metric_name}')
    plt.xlabel('Epoch')
    plt.ylabel(metric_name)
    plt.legend(loc='lower right')
    plt.plot()

In [ ]:
# Plots the results of a time series.
# model is the neural network
# generator_to_evaluate is either the generator trained on your training or testing set
# scaler is the min-max scaler trained in §2.
def plot_time_series_results(model, generator_to_evaluate, scaler, dates_index):

    # Returns pandas dataframe that contains (i) actual value (milk production) and (ii) predicted value. Both descaled.
    def get_results_df():
        
        # Creates list to store  (i) predicted and (ii) actual values
        all_predictions = []
        all_actuals = []

        for i in range(len(generator_to_evaluate)):
            # Gets a batch of (i) X features and (i) the target they're trying to predict
            x_batch, y_batch = generator_to_evaluate[i]

            # Make predictions on the current batch
            batch_predictions = model.predict(x_batch, verbose=0)

            # Extend our lists with the current batch's predictions and actuals
            # Flatten them if they are in shape (batch_size, 1) to (batch_size,)
            all_predictions.extend(batch_predictions.flatten())
            all_actuals.extend(y_batch.flatten())

        # Convert lists to NumPy arrays for easier manipulation
        all_predictions = np.array(all_predictions).reshape(-1, 1) # Reshape back to (n_samples, 1) for inverse_transform
        all_actuals = np.array(all_actuals).reshape(-1, 1)

        # Descales predictions (via the scaler) so they're intelligible again (i.e., not approximately 0-1)
        all_predictions = scaler.inverse_transform(all_predictions)
        all_actuals = scaler.inverse_transform(all_actuals)


        # Create a DataFrame for easy viewing
        results_df = pd.DataFrame({'Actual': all_actuals.flatten(), 'Predicted': all_predictions.flatten()},
                                 index = dates_index[n_input:])
        return results_df

    # Plots the results df. Takes the results_df returned in the above subfunction.
    def plot_results_df(results_df):

        # Defines plot size
        plt.figure(figsize=(7, 5))

        # Plots vals
        plt.plot(results_df['Actual'], label='Actual Values (Test Set)', color='blue', linewidth=2, marker='o', markersize=4)
        plt.plot(results_df['Predicted'], label='Predicted Values (Test Set)', color='red', linestyle='--', linewidth=1.5, marker='o', markersize=4)

        
        # Labels
        plt.title('LSTM Model Predictions vs. Actuals')
        plt.xlabel('Date')
        plt.ylabel('Mikl Production (Original Scale)')

        # Other
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        
    # Main functions
    results_df = get_results_df()
    plot_results_df(results_df)
    return results_df

### 1.3 | Loads & Inspects Data

In [ ]:
# Now load the data using the pandas dataframe. We will use milk production data
df = pd.read_csv('res/lstm/monthly_milk_production.csv',
                 index_col='Date',
                 parse_dates=True)
df.index.freq = 'MS'

In [ ]:
# Inspects dataset
print(df.shape)
df.head(3)

In [ ]:
# Plotting graph: production and date
df.plot(figsize=(12, 6))

In [ ]:
# Same as above, except it plots seasonality, trends, and noise.
# Noise is defined as time series datapoint - trend - seasonality
seasonal_decompose(df['Production']).plot()

# 2 | Preprocessing

### 2.1 | Train - Test Split

In [ ]:
# Splitting the data into training and testing.
# We're not using scikit learn's train/test split becuase we want the most recent years to be forecasted

train = df.iloc[:156] 
test = df.iloc[156:] 

print(f"Training set size: {train.shape[0]} months")
print(f"Training set size: {test.shape[0]} months")

### 2.2 | Scaling

Neural networks are extremely sensitive to large values. We'll need to convert the numeric features (milk production) into something a neural network can handle, like a 0-1 value.

In [ ]:
# These numbers (milk production) are too large for a neural network to effectively handle.
train["Production"]

In [ ]:
# Defines a min-max scaler, in which all values are scaled between the minimum value and the max value
# See here for formula: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
scaler = MinMaxScaler()

In [ ]:
# Fits the scaler's parameters on the training data (e.g., x min, x max)
scaler.fit(train)

In [ ]:
# Scales the training & test data
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

# Ensures scaler worked
print(f"min milk value before scaling = {min(train['Production'])} | with scaling = {min(scaled_train)}")
print(f"max milk value after scaling = {max(train['Production'])} | with scaling = {max(scaled_train)}")

In [ ]:
# Prints results
print("First three scaled values")
print(scaled_train[:3])

### 2.3 | Converts our data to a format conducive to LSTMs

LSTMs take a sequence. In other words, we feed it X dates, and the LSTM will predict the (X+1) date. In order to feed the LSTM sequences, then, we need to use the "TimeSeriesGenerator" to convert our data into sequences.

In [ ]:

# Creates the training & testing set.
n_input = 3 # number of input months
generator_train = TimeseriesGenerator(data = scaled_train, targets = scaled_train,
                                      length = n_input, batch_size=32)
generator_test = TimeseriesGenerator(data = scaled_test, targets = scaled_test,
                                      length = n_input, batch_size=32)

In [ ]:
# Displays the raw input & output
X, y = generator_train[0]

print("Here is how the neural network will work.\n")

print(f'Given the Array: \n{X[0]}')
print(f'Predict this y: \n {y[0]}')

print("\nKeep in mind these values are SCALED. Here's what the unscaled looks like")
print("Here's the X: ")
print(scaler.inverse_transform(X[0]))
print("Here's the y (the next month's milk production):")
print(scaler.inverse_transform([y[0]]))

# 3 | LSTM

### 3.0 | Section Overview

We'll create a neural network of LSTM cells (§3.1) before plotting the output (§3.2) There's a few things to note. 

When creating the LSTM, Keras keeps its implementation to a very high level such that you don't need to know the specific inner-workings of a LSTM cell. However, there's a few parameters to note.
<ul>
  <li> <strong>activation.</strong> This is the activation function that's responsible for creating the new memory. This activation function occurs twice per cell: in the input gate (long term memory) and the output gate (short term memory). </li>
  <li> <strong>recurrent activation.</strong> This is the activation function that's responsible for the percentage of memory to remember. This activation function occurs thrice per cell: in the forgotten gate (updating the long term memory), in the input gate (long term memory) and the output gate (short term memory). </li>
    <li> <strong>return sequence</strong> By default, this is False. Keras sets up an LSTM layer such that the input passes through EACH NEURON in the layer. For example, if there are 64 neurons, the input will pass from LSTM cell #1, then to cell #2, etc. Sometimes, however, you want multiple LSTM layers running 'in parrallel' to each other. To implement this functionality, set the return_sequences = True until you get to the final LSTM layer. </li>
</ul>

When evaluating the model, please note this is not a classification problem. There are no 'neat' precision, recall, and f1 values. Instead, there's mean squared error, which is difficult to interpret. However, we can plot the predicted values alongside the actual values to assess the model's utility.

### 3.1 | LSTM: Construct & Train

In [ ]:
# Creates model
initial_lstm_neural_network = Sequential()
initial_lstm_neural_network.add( Input( shape= (n_input,1) ) )  # Input layer. shape refers to (number_of_inputs, number_of_features)
initial_lstm_neural_network.add(LSTM(128, activation="tanh", recurrent_activation="sigmoid", return_sequences = True)) 
initial_lstm_neural_network.add(LSTM(64, activation="tanh", recurrent_activation="sigmoid",)) 
initial_lstm_neural_network.add(Dense(1, activation = "linear", )) # Linear activation because our output is a number (c.f., sigmoid for binary yes/no)

# Compiles model
initial_lstm_neural_network.compile(loss='mse', optimizer=Adam(learning_rate=.001), 
             metrics=[metrics.MeanSquaredError(name='mse'),])
initial_lstm_neural_network.summary()

In [ ]:
hist = initial_lstm_neural_network.fit(generator_train, validation_data =generator_test, epochs=100)

### 3.2 | Results 

In [ ]:
loss, val_loss = hist.history["loss"], hist.history["val_loss"]
print(f"Total Training Loss = {hist.history['loss'][-1]} | Testing Loss = {hist.history['val_loss'][-1]}")
plot_performance(loss, val_loss, "Loss")

In [ ]:
# Training set
_ = plot_time_series_results(initial_lstm_neural_network, generator_train, scaler, train.index)

In [ ]:
# Testing set
_ = plot_time_series_results(initial_lstm_neural_network, generator_test, scaler, test.index)

# 4 | Your Turn

### 4.0 | Section Overview

Improve upon the previous LSTM. You can add more hidden layers, change the number of nodes, change the activation function to something like relu, etc.

To determine if your model is better than the previous, your model should have a smaller loss function on the testing (validation) set.

<strong> Do not change the loss function </strong>

In [ ]:
# Just run this method
def did_you_win(your_loss, my_loss, silence = False):

    # A neural network may perform better due to random chance. 
    # Thus, we'll use a 'tolerance' value. The model will not be considered better unless it exceeds the tolerance.
    TOLERANCE = 0.01
    you_lost = your_loss >= (my_loss-TOLERANCE)
    
    if you_lost:
        print(f"{'*'*20}\nYou lost.\n{'*'*20}\nYour loss is greater than, equal to, or not significantly beter than my loss.")
        print(f"Your neural network's loss must be {TOLERANCE} less than my neural network's loss to be considered better.")
        print("This is because your neural network may be doing better just due to random chance")
        print("This can be done by (i) improving fitting and (ii) combatting overfitting, if any exists.")
    else:
        print("Congratulations! You won! Your model outperforms my model on the testing set!")
        
    print(f"\nYour Loss = [{your_loss}] | My loss = {my_loss} | Loss Difference (Negative means your model does better) = {your_loss - my_loss}")

### 4.1 | Create your model 

In [ ]:
# Creates model
your_model = Sequential()

# Input Layer
your_model.add( Input( shape= (n_input,1) ) ) 

# Hidden Layers
your_model.add(LSTM(128, activation="tanh", recurrent_activation="sigmoid", return_sequences = True)) 
your_model.add(LSTM(64, activation="tanh", recurrent_activation="sigmoid",)) 

# Output Layer
your_model.add(Dense(1, activation = "linear", ))

# Compiles model
your_model.compile(loss='mse', optimizer=Adam(learning_rate=.001), 
             metrics=[metrics.MeanSquaredError(name='mse'),])
your_model.summary()

In [ ]:
# Train your model
your_hist = your_model.fit(generator_train, validation_data = generator_test, epochs=100)

### 4.2 | Your Results

In [ ]:
# Prints your results
print(f"Your Training Loss = {your_hist.history['loss'][-1]} | Your Testing Loss = {your_hist.history['val_loss'][-1]}")

In [ ]:
# Let's see if you won. Just run this code. DO NOT MODIFY IT
did_you_win(your_loss = your_hist.history['val_loss'][-1], my_loss = hist.history['val_loss'][-1])

### 4.3 | Your Graphs (Optional)

In [ ]:
loss, val_loss = your_hist.history["loss"], your_hist.history["val_loss"]
plot_performance(loss, val_loss, "Loss")

In [ ]:
# Training set
_ = plot_time_series_results(your_model, generator_train, scaler, train.index)

In [ ]:
# Testing set
_ = plot_time_series_results(your_model, generator_test, scaler, test.index)